In [ ]:
%pip install requests orjson

In [18]:
import json
import orjson
import os
import requests
import time


### 각종 json에 필요없는 요소를 제거하는 함수입니다.
def data_cleaning(data):
    data = data['data']
    ### myData 요소를 제거합니다.
    for d in data:
        del d['myData']

    return data


def write_json(data, output_dir):
    for summoner_id, match in data:
        with open(os.path.join(output_dir, f"{summoner_id}.json"), 'wb') as file:
            file.write(orjson.dumps(match))
            

def summoner_id_generator(summoner_id_path, output_dir):
    num_of_file = len(os.listdir(output_dir))

    with open(summoner_id_path) as json_file:
        json_data = json.load(json_file)
        for i in json_data[num_of_file:]:
            summoner_id = i['summoner_id']
            yield summoner_id


def crawling_match_generator(headers, summoner_id_generator):
    for summoner_id in summoner_id_generator:
        # print(f"start: {summoner_id}")
        url = f"https://www.op.gg/api/v1.0/internal/bypass/games/kr/summoners/{summoner_id}?&limit=10&hl=ko_KR&game_type=soloranked"
        resp = requests.get(url, headers=headers)

        if resp.status_code == 200:
            user_json = orjson.loads(resp.text)
            user_json = data_cleaning(user_json)
            yield summoner_id, user_json
            
        elif resp.status_code == 404:
            user_json = orjson.loads('[{"resp.status_code": 404}]')
            yield summoner_id, user_json

        ### 응답코드가 429일 때, 1초씩 늘려가며 다시 요청합니다.
        elif resp.status_code == 429:
            sleep_time = 1
            while resp.status_code == 429:
                time.sleep(sleep_time)
                sleep_time += 1

                print(f'sleep {sleep_time}')      
                resp = requests.get(url, headers=headers)
        
        else:
            print(f"unexpected response : {resp.status_code}, summoncer_id : {summoner_id}")

        

In [19]:
if __name__ == "__main__":
    tier = 'platinum'
    summoner_id_path = "../boostcamp_notebooks/summorer_ids"
    output_dir = "/home/piddle/hdd/matches"
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:122.0) Gecko/20100101 Firefox/122.0',}
    
    target_dir = os.path.join(output_dir, tier)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        
    summoner_id_gen = summoner_id_generator(os.path.join(summoner_id_path, tier + '.json'), target_dir)
    match_gen = crawling_match_generator(headers, summoner_id_gen)
    write_json(match_gen, target_dir)

##### 유저별 매치개수를 10개로 변경하고, json 문법의 들여쓰기를 제거하는 코드입니다.

In [ ]:
tier = 'diamond'
output_dir = "/home/piddle/hdd/matches"
path = os.path.join(output_dir, tier)


for file in os.listdir(path):
    # print(f'start: {file}')
    file_path = os.path.join(path , file)
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)

        data = data[:10]
        
        try:
            for d in data:
                del d['myData']

        except:
            print('not found \'myData\'')

        json_file = open(file_path, 'w')
        json.dump(data, json_file, ensure_ascii=False)
